In [35]:
from tinyman.v1.client import TinymanMainnetClient

In [ ]:
import algosdk
#private_key=algosdk.mnemonic.to_private_key('list of 25 words from wallet')

In [54]:
account = {
    'address': algorand_wallet_address,
    'private_key':base64_private_key # Use algosdk.mnemonic.to_private_key(mnemonic) if necessary
}

In [55]:
client = TinymanMainnetClient(user_address=account['address'])

In [57]:

# Check if the account is opted into Tinyman and optin if necessary
if(not client.is_opted_in()):
    print('Account not opted into app, opting in now..')
    transaction_group = client.prepare_app_optin_transactions()
    transaction_group.sign_with_private_key(account['address'], account['private_key'])
    result = client.submit(transaction_group, wait=True)

In [60]:
USDC = client.fetch_asset(31566704)
ALGO = client.fetch_asset(0)

In [61]:
# Fetch the pool we will work with
pool = client.fetch_pool(USDC, ALGO)

In [62]:
# Get a quote for a swap of 1 ALGO to TINYUSDC with 1% slippage tolerance
quote = pool.fetch_fixed_input_swap_quote(ALGO(1_000_000), slippage=0.01)
print(quote)
print(f'USDC per ALGO: {quote.price}')
print(f'USDC per ALGO (worst case): {quote.price_with_slippage}')

SwapQuote(swap_type='fixed-input', amount_in=ALGO('1'), amount_out=USDC('1.813101'), swap_fees=ALGO('0.003'), slippage=0.01)
USDC per ALGO: 1.813101
USDC per ALGO (worst case): 1.79497


In [82]:
print(pool.fetch_fixed_input_swap_quote(ALGO(50000000.0), slippage=0.03).price_with_slippage)

1.77922832


In [63]:
# We only want to sell if ALGO is > 1.70 TINYUSDC
if quote.price_with_slippage > 1.70:
    print(f'Swapping {quote.amount_in} to {quote.amount_out_with_slippage}')
    # Prepare a transaction group
    transaction_group = pool.prepare_swap_transactions_from_quote(quote)
    # Sign the group with our key
    transaction_group.sign_with_private_key(account['address'], account['private_key'])
    # Submit transactions to the network and wait for confirmation
    result = client.submit(transaction_group, wait=True)

    # Check if any excess remaining after the swap
    excess = pool.fetch_excess_amounts()
    if USDC in excess:
        amount = excess[USDC]
        print(f'Excess: {amount}')
        # We might just let the excess accumulate rather than redeeming if its < 0.005 USDC
        if amount > 0.005:
            transaction_group = pool.prepare_redeem_transactions(amount)
            transaction_group.sign_with_private_key(account['address'], account['private_key'])
            result = client.submit(transaction_group, wait=True)

Swapping ALGO('1') to USDC('1.79497')
Waiting for confirmation
Transaction EDTGOFRZR454TLF7IQHUX42BO6Y63UAYDYCMFFMJCEXPVVRE3W6Q confirmed in round 17286838.
Excess: USDC('0.018131')
Waiting for confirmation
Transaction 3ISU4X7DA3ILF3Y6IXEV7PXFEIOSFFABFHTE3GXD5Z2GMZAXTPIQ confirmed in round 17286840.
